In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sklm
import xgboost as xgb
import matplotlib
import lightgbm as lgb

In [2]:
train = pd.read_csv('Train.csv', parse_dates=['Date'])
test = pd.read_csv('Test.csv', parse_dates=['Date'])
submissions = pd.read_csv('SampleSubmission.csv')

In [3]:
print(train.shape)
print(test.shape)

(30557, 82)
(16136, 77)


In [4]:
traincols = list(train.columns)
testcols = list(test.columns)
#print(traincols)
#print(testcols)

In [5]:
# Look at correlation with target
corr = dict(train.corr()['target'].sort_values())

In [6]:
Labels = np.array(train['target'])
print(Labels)
#train = train.drop(['target'], axis=1)

[ 38.  39.  24. ...  85. 103.  89.]


In [7]:
extracted_columns = ['relative_humidity_2m_above_ground', 'specific_humidity_2m_above_ground', 'temperature_2m_above_ground',
                     'u_component_of_wind_10m_above_ground', 'v_component_of_wind_10m_above_ground', 'L3_NO2_NO2_column_number_density',
                     'L3_NO2_NO2_slant_column_number_density', 'L3_NO2_sensor_altitude', 'L3_NO2_tropospheric_NO2_column_number_density',
                     'L3_O3_O3_column_number_density', 'L3_CO_CO_column_number_density', 'L3_CO_solar_zenith_angle', 
                    'L3_CLOUD_cloud_optical_depth', 'L3_CLOUD_surface_albedo', 'L3_AER_AI_absorbing_aerosol_index', 'L3_AER_AI_sensor_altitude',
                     'day', 'Place_ID', 'month']

In [8]:
train['day'] = train.Date.dt.day
test['day'] = test.Date.dt.day

train['month'] = train.Date.dt.month
test['month'] = test.Date.dt.month

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Place_ID'] = le.fit_transform(train['Place_ID'])
test['Place_ID'] = le.fit_transform(test['Place_ID'])

In [10]:
for i in testcols:
    if i not in extracted_columns:
        test = test.drop(i, axis=1)
for j in traincols:
    if j not in extracted_columns:
        train = train.drop(j, axis=1)

In [11]:
print(train.shape)
print(test.shape)

(30557, 19)
(16136, 19)


In [12]:
Features = np.array(train)
test_enc = np.array(test)

import numpy.random as nr
import sklearn.model_selection as ms
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [13]:
lg = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.1, n_estimators=2000, max_depth=14)
eval_set = [(x_test, y_test)]
lg.fit(x_train, y_train, eval_set=eval_set, eval_metric="rmse")

[1]	valid_0's rmse: 45.8511	valid_0's l2: 2102.32
[2]	valid_0's rmse: 44.5276	valid_0's l2: 1982.71
[3]	valid_0's rmse: 43.3519	valid_0's l2: 1879.39
[4]	valid_0's rmse: 42.3346	valid_0's l2: 1792.22
[5]	valid_0's rmse: 41.3835	valid_0's l2: 1712.59
[6]	valid_0's rmse: 40.6037	valid_0's l2: 1648.66
[7]	valid_0's rmse: 39.8823	valid_0's l2: 1590.6
[8]	valid_0's rmse: 39.227	valid_0's l2: 1538.76
[9]	valid_0's rmse: 38.6833	valid_0's l2: 1496.4
[10]	valid_0's rmse: 38.1681	valid_0's l2: 1456.8
[11]	valid_0's rmse: 37.6948	valid_0's l2: 1420.9
[12]	valid_0's rmse: 37.2989	valid_0's l2: 1391.21
[13]	valid_0's rmse: 36.8601	valid_0's l2: 1358.66
[14]	valid_0's rmse: 36.4917	valid_0's l2: 1331.64
[15]	valid_0's rmse: 36.1531	valid_0's l2: 1307.05
[16]	valid_0's rmse: 35.8182	valid_0's l2: 1282.95
[17]	valid_0's rmse: 35.451	valid_0's l2: 1256.77
[18]	valid_0's rmse: 35.2274	valid_0's l2: 1240.97
[19]	valid_0's rmse: 34.9748	valid_0's l2: 1223.24
[20]	valid_0's rmse: 34.7534	valid_0's l2: 120

[162]	valid_0's rmse: 28.3138	valid_0's l2: 801.669
[163]	valid_0's rmse: 28.3099	valid_0's l2: 801.45
[164]	valid_0's rmse: 28.3058	valid_0's l2: 801.219
[165]	valid_0's rmse: 28.2924	valid_0's l2: 800.457
[166]	valid_0's rmse: 28.2857	valid_0's l2: 800.08
[167]	valid_0's rmse: 28.2772	valid_0's l2: 799.602
[168]	valid_0's rmse: 28.2727	valid_0's l2: 799.346
[169]	valid_0's rmse: 28.2666	valid_0's l2: 799.002
[170]	valid_0's rmse: 28.2441	valid_0's l2: 797.727
[171]	valid_0's rmse: 28.2319	valid_0's l2: 797.04
[172]	valid_0's rmse: 28.228	valid_0's l2: 796.819
[173]	valid_0's rmse: 28.2189	valid_0's l2: 796.308
[174]	valid_0's rmse: 28.2079	valid_0's l2: 795.684
[175]	valid_0's rmse: 28.2049	valid_0's l2: 795.515
[176]	valid_0's rmse: 28.1957	valid_0's l2: 794.997
[177]	valid_0's rmse: 28.1918	valid_0's l2: 794.777
[178]	valid_0's rmse: 28.1893	valid_0's l2: 794.634
[179]	valid_0's rmse: 28.18	valid_0's l2: 794.111
[180]	valid_0's rmse: 28.1775	valid_0's l2: 793.973
[181]	valid_0's rm

[321]	valid_0's rmse: 27.4126	valid_0's l2: 751.451
[322]	valid_0's rmse: 27.4088	valid_0's l2: 751.244
[323]	valid_0's rmse: 27.411	valid_0's l2: 751.364
[324]	valid_0's rmse: 27.4117	valid_0's l2: 751.402
[325]	valid_0's rmse: 27.4109	valid_0's l2: 751.359
[326]	valid_0's rmse: 27.4108	valid_0's l2: 751.354
[327]	valid_0's rmse: 27.4042	valid_0's l2: 750.99
[328]	valid_0's rmse: 27.4035	valid_0's l2: 750.951
[329]	valid_0's rmse: 27.4011	valid_0's l2: 750.822
[330]	valid_0's rmse: 27.3997	valid_0's l2: 750.741
[331]	valid_0's rmse: 27.3945	valid_0's l2: 750.456
[332]	valid_0's rmse: 27.3939	valid_0's l2: 750.426
[333]	valid_0's rmse: 27.3968	valid_0's l2: 750.584
[334]	valid_0's rmse: 27.3903	valid_0's l2: 750.23
[335]	valid_0's rmse: 27.3841	valid_0's l2: 749.89
[336]	valid_0's rmse: 27.3819	valid_0's l2: 749.767
[337]	valid_0's rmse: 27.3784	valid_0's l2: 749.577
[338]	valid_0's rmse: 27.3754	valid_0's l2: 749.414
[339]	valid_0's rmse: 27.3679	valid_0's l2: 749.004
[340]	valid_0's 

[483]	valid_0's rmse: 26.989	valid_0's l2: 728.405
[484]	valid_0's rmse: 26.9816	valid_0's l2: 728.009
[485]	valid_0's rmse: 26.9758	valid_0's l2: 727.696
[486]	valid_0's rmse: 26.9832	valid_0's l2: 728.091
[487]	valid_0's rmse: 26.9847	valid_0's l2: 728.174
[488]	valid_0's rmse: 26.9875	valid_0's l2: 728.323
[489]	valid_0's rmse: 26.9853	valid_0's l2: 728.206
[490]	valid_0's rmse: 26.9716	valid_0's l2: 727.467
[491]	valid_0's rmse: 26.971	valid_0's l2: 727.436
[492]	valid_0's rmse: 26.9678	valid_0's l2: 727.262
[493]	valid_0's rmse: 26.9648	valid_0's l2: 727.101
[494]	valid_0's rmse: 26.9652	valid_0's l2: 727.121
[495]	valid_0's rmse: 26.9634	valid_0's l2: 727.025
[496]	valid_0's rmse: 26.9585	valid_0's l2: 726.761
[497]	valid_0's rmse: 26.953	valid_0's l2: 726.463
[498]	valid_0's rmse: 26.9522	valid_0's l2: 726.42
[499]	valid_0's rmse: 26.9505	valid_0's l2: 726.328
[500]	valid_0's rmse: 26.94	valid_0's l2: 725.762
[501]	valid_0's rmse: 26.9357	valid_0's l2: 725.534
[502]	valid_0's rm

[642]	valid_0's rmse: 26.7233	valid_0's l2: 714.135
[643]	valid_0's rmse: 26.7163	valid_0's l2: 713.758
[644]	valid_0's rmse: 26.7176	valid_0's l2: 713.828
[645]	valid_0's rmse: 26.7178	valid_0's l2: 713.841
[646]	valid_0's rmse: 26.7172	valid_0's l2: 713.809
[647]	valid_0's rmse: 26.7192	valid_0's l2: 713.918
[648]	valid_0's rmse: 26.7148	valid_0's l2: 713.681
[649]	valid_0's rmse: 26.7131	valid_0's l2: 713.591
[650]	valid_0's rmse: 26.7042	valid_0's l2: 713.115
[651]	valid_0's rmse: 26.702	valid_0's l2: 712.998
[652]	valid_0's rmse: 26.7009	valid_0's l2: 712.939
[653]	valid_0's rmse: 26.7008	valid_0's l2: 712.931
[654]	valid_0's rmse: 26.7007	valid_0's l2: 712.93
[655]	valid_0's rmse: 26.6995	valid_0's l2: 712.862
[656]	valid_0's rmse: 26.7001	valid_0's l2: 712.894
[657]	valid_0's rmse: 26.7	valid_0's l2: 712.89
[658]	valid_0's rmse: 26.6936	valid_0's l2: 712.546
[659]	valid_0's rmse: 26.6958	valid_0's l2: 712.664
[660]	valid_0's rmse: 26.696	valid_0's l2: 712.679
[661]	valid_0's rms

...

[1434]	valid_0's rmse: 26.175	valid_0's l2: 685.133
[1435]	valid_0's rmse: 26.1747	valid_0's l2: 685.116
[1436]	valid_0's rmse: 26.1748	valid_0's l2: 685.121
[1437]	valid_0's rmse: 26.1757	valid_0's l2: 685.168
[1438]	valid_0's rmse: 26.1759	valid_0's l2: 685.177
[1439]	valid_0's rmse: 26.1756	valid_0's l2: 685.163
[1440]	valid_0's rmse: 26.1739	valid_0's l2: 685.073
[1441]	valid_0's rmse: 26.1738	valid_0's l2: 685.067
[1442]	valid_0's rmse: 26.1729	valid_0's l2: 685.02
[1443]	valid_0's rmse: 26.1727	valid_0's l2: 685.008
[1444]	valid_0's rmse: 26.1748	valid_0's l2: 685.119
[1445]	valid_0's rmse: 26.1741	valid_0's l2: 685.081
[1446]	valid_0's rmse: 26.1743	valid_0's l2: 685.093
[1447]	valid_0's rmse: 26.174	valid_0's l2: 685.08
[1448]	valid_0's rmse: 26.1739	valid_0's l2: 685.071
[1449]	valid_0's rmse: 26.1733	valid_0's l2: 685.042
[1450]	valid_0's rmse: 26.1732	valid_0's l2: 685.035
[1451]	valid_0's rmse: 26.1734	valid_0's l2: 685.047
[1452]	valid_0's rmse: 26.1743	valid_0's l2: 685.0

[1590]	valid_0's rmse: 26.148	valid_0's l2: 683.72
[1591]	valid_0's rmse: 26.1476	valid_0's l2: 683.699
[1592]	valid_0's rmse: 26.147	valid_0's l2: 683.666
[1593]	valid_0's rmse: 26.1462	valid_0's l2: 683.622
[1594]	valid_0's rmse: 26.1454	valid_0's l2: 683.581
[1595]	valid_0's rmse: 26.145	valid_0's l2: 683.561
[1596]	valid_0's rmse: 26.1449	valid_0's l2: 683.554
[1597]	valid_0's rmse: 26.1455	valid_0's l2: 683.59
[1598]	valid_0's rmse: 26.1425	valid_0's l2: 683.431
[1599]	valid_0's rmse: 26.1427	valid_0's l2: 683.441
[1600]	valid_0's rmse: 26.1405	valid_0's l2: 683.325
[1601]	valid_0's rmse: 26.1407	valid_0's l2: 683.336
[1602]	valid_0's rmse: 26.1398	valid_0's l2: 683.288
[1603]	valid_0's rmse: 26.1398	valid_0's l2: 683.289
[1604]	valid_0's rmse: 26.1397	valid_0's l2: 683.282
[1605]	valid_0's rmse: 26.14	valid_0's l2: 683.3
[1606]	valid_0's rmse: 26.1404	valid_0's l2: 683.323
[1607]	valid_0's rmse: 26.1401	valid_0's l2: 683.305
[1608]	valid_0's rmse: 26.138	valid_0's l2: 683.197
[16

[1747]	valid_0's rmse: 26.1306	valid_0's l2: 682.81
[1748]	valid_0's rmse: 26.1308	valid_0's l2: 682.821
[1749]	valid_0's rmse: 26.131	valid_0's l2: 682.831
[1750]	valid_0's rmse: 26.131	valid_0's l2: 682.831
[1751]	valid_0's rmse: 26.1321	valid_0's l2: 682.886
[1752]	valid_0's rmse: 26.132	valid_0's l2: 682.884
[1753]	valid_0's rmse: 26.132	valid_0's l2: 682.879
[1754]	valid_0's rmse: 26.1309	valid_0's l2: 682.825
[1755]	valid_0's rmse: 26.1311	valid_0's l2: 682.832
[1756]	valid_0's rmse: 26.1308	valid_0's l2: 682.82
[1757]	valid_0's rmse: 26.131	valid_0's l2: 682.828
[1758]	valid_0's rmse: 26.1306	valid_0's l2: 682.81
[1759]	valid_0's rmse: 26.1313	valid_0's l2: 682.842
[1760]	valid_0's rmse: 26.1302	valid_0's l2: 682.787
[1761]	valid_0's rmse: 26.1302	valid_0's l2: 682.789
[1762]	valid_0's rmse: 26.1301	valid_0's l2: 682.783
[1763]	valid_0's rmse: 26.1306	valid_0's l2: 682.81
[1764]	valid_0's rmse: 26.1304	valid_0's l2: 682.797
[1765]	valid_0's rmse: 26.1281	valid_0's l2: 682.68
[17

[1903]	valid_0's rmse: 26.1078	valid_0's l2: 681.619
[1904]	valid_0's rmse: 26.1084	valid_0's l2: 681.651
[1905]	valid_0's rmse: 26.1085	valid_0's l2: 681.656
[1906]	valid_0's rmse: 26.1079	valid_0's l2: 681.62
[1907]	valid_0's rmse: 26.1079	valid_0's l2: 681.624
[1908]	valid_0's rmse: 26.1072	valid_0's l2: 681.583
[1909]	valid_0's rmse: 26.1076	valid_0's l2: 681.607
[1910]	valid_0's rmse: 26.1079	valid_0's l2: 681.62
[1911]	valid_0's rmse: 26.1082	valid_0's l2: 681.64
[1912]	valid_0's rmse: 26.1082	valid_0's l2: 681.639
[1913]	valid_0's rmse: 26.1072	valid_0's l2: 681.584
[1914]	valid_0's rmse: 26.107	valid_0's l2: 681.576
[1915]	valid_0's rmse: 26.1068	valid_0's l2: 681.567
[1916]	valid_0's rmse: 26.1069	valid_0's l2: 681.569
[1917]	valid_0's rmse: 26.1056	valid_0's l2: 681.501
[1918]	valid_0's rmse: 26.1055	valid_0's l2: 681.497
[1919]	valid_0's rmse: 26.1057	valid_0's l2: 681.509
[1920]	valid_0's rmse: 26.1061	valid_0's l2: 681.53
[1921]	valid_0's rmse: 26.1064	valid_0's l2: 681.54

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=14,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [14]:
predictions = lg.predict(x_test)
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, predictions) ** 0.5

26.090155816230826

In [15]:
preds = lg.predict(test_enc)
submissions.target = preds
submissions.to_csv('LGGGGG.csv', index=False)